```python
# Copyright 2022 Bloomberg Finance L.P.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

# Market Value Item

### What is market value?

All of the previously discussed items have focused on different collections of securities (positions, accounts, and portfolios). Now we're going to take another step forward and discuss market value. Market value is generally the price that an asset of any type (e.g., equity, fixed income, derivatives, fx) would fetch in the marketplace. It is also the total value of the securities in which you hold a position. Depending on the security type, market value can be simple (i.e., for equities, the value of a unit is well-known) or more complex with various options (i.e., for derivatives, there are a few ways to determine and/or state what the market value of your position is).

For our purpose we'll stick with simple market value calculations where market value is equal to some unit multiplier times your positions

For example, you could have a position and current share price/unit market value like below:

My position is ***2000*** shares of ***MSFT USD***. The current price of ***MSFT USD*** is ***19.00***. This gives my position a 
market value of **38,000 USD**!

### Problem Definition

We want to extend our classes to allow for querying of the market values for various securities at different levels of aggregation. We want this classes to have to the ability below functionality:

***Security***
- Query the current market value of the security

***Positions*** 
- Query the current market value for the current position

***Account***
- Query the market value for a given set of security names or security objects. These objects should act as filters to what should be included in calculating the accounts net market value. Any securities not present in the account but the filter can be ignored
- Query the market value for the entire account. This should return a net market value of all positions in the account.

***Portfolio***
- Query the market value for the entire portfolio. This requires that security market values be atomic in your calculation method per query.
- Query the market value for a given set of securities/account combinations. If securities are present they should act as a filter on the accounts to be queried. If accounts are present they should act as a filter on the accounts under query. Handle scenarios where securities or accounts given aren't present without causing exceptions.
- **Create tests for the above portfolio features**

### Provided Tools

#### *Data Source*

To simulate prices, you can utilize the given class priceData which can be imported from generators.priceDataGenerator. This class is a singleton and will generate prices each time a query is given. These prices are stored internally in the class for testing in a replay mode. For a given security if the security name contains "Equity" or "eqty" (case insensitive) the market value will always be positive. To query the price of a security you can use the object's getCurrentPrice method. Below is an example of this

```python
#Data Souce
from generators.priceDataGenerator import priceData

pD = priceData()

curPrice = pD.getCurrentPrice("IBM US Equity")
```

#### *Solution Interface*

The below interface show the new functionality you should add to your existing classes.

```python
#filename securityInterface.py
#Security Class Interface
class securityInterface():
    #...
    #Previous implementation above
    def getCurrentMarketValue(self) -> float:
```

```python
#filename positionInterface.py
#Position Class Interface
class positionInterface():
    #...
    #Previous implementation above
    def getCurrentMarketValue(self) -> float:
        pass
```

```python
#filename accountInterface.py
#Account Class Interface
class accountInterface():
    #...
    #Previous implementation above
    
    def getCurrentMarketValue(self) -> float:
        pass

    def getCurrentFilteredMarketValue(self, securities: Set) -> float:
        pass
```

```python
#filename portfolioInterface.py
#Portfolio Class Interface
class portfolioInterface():
    #...
    #Previous implementation above
    
    def getCurrentMarketValue(self) -> dict:
        pass

    def getCurrentFilteredMarketValue(self, securities: Set, accountNames: Set[str]) -> float:
        pass
```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to files to their respective .py's in the implementation folders. **Remember to copy the current implementation as saving will overwrite your previous implementation!** 

In [ ]:
#%%writefile ../implementations/securitySolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements securityInterface & allows for the management of a security

In [ ]:
#%%writefile ../implementations/positionSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements positionInterface & allows for the management of a position

In [ ]:
#%%writefile ../implementations/accountSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements accountInterface & allows for the management of an account

In [ ]:
#%%writefile ../implementations/portfolioSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements portFolioInterface & allows for the management of a portfolio

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq
import pytest
import implementations.securitySolution
import implementations.positionSolution
import implementations.accountSolution
from generators.priceDataGenerator import priceData
import importlib
importlib.reload(implementations.securitySolution)
importlib.reload(implementations.positionSolution)
importlib.reload(implementations.accountSolution)

def test_securityValueGather():
    #GIVEN
    SECURITY_NAME = "TSLA US Equity"
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()

    #WHEN
    testObj = implementations.securitySolution.security(SECURITY_NAME)
    currentPrice = testObj.getCurrentMarketValue()

    #EXPECT
    assert currentPrice == DATA_SOURCE.getSecurityPriceDataList(SECURITY_NAME)[-1]

def test_PositionMarketValue():
    #GIVEN
    EXPECTED_NAME = "IBM US Equity"
    EXPECTED_POSITION_AMOUNT = 1000
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()

    #WHEN
    testObj = implementations.positionSolution.position(EXPECTED_NAME, EXPECTED_POSITION_AMOUNT)
    MV = testObj.getCurrentMarketValue()
    LASTEST_EXPECTED_MV = EXPECTED_POSITION_AMOUNT * DATA_SOURCE.getSecurityPriceDataList(EXPECTED_NAME)[-1]

    #EXPECT
    assert (LASTEST_EXPECTED_MV == MV)

def test_SecuritySearchAccountMV():
    #GIVEN
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()
    EXPECTED_ACCOUNT_POSITIONS = [
        implementations.positionSolution.position("IBM US Equity", 530),
        implementations.positionSolution.position("TSLA US Equity", 1120),
        implementations.positionSolution.position("NVDA US Equity", 7421)
    ]
    SEARCH_SECURITIES_LIST =  ["IBM US Equity", implementations.securitySolution.security("NVDA US Equity"), "MSFT US Equity"]
    SEARCH_SECURITIES_TUPLE = [["IBM US Equity", 530], ["NVDA US Equity", 7421]]
    testObj = implementations.accountSolution.account(EXPECTED_ACCOUNT_POSITIONS, "Test Account")

    #WHEN
    MV = testObj.getCurrentFilteredMarketValue(SEARCH_SECURITIES_LIST)
    EXPECTED_MV = 0
    for secTuple in SEARCH_SECURITIES_TUPLE:
        if secTuple[1] != 0:
            EXPECTED_MV +=  secTuple[1] * DATA_SOURCE.getSecurityPriceDataList(secTuple[0])[-1]
    #EXPECT
    assert (EXPECTED_MV == MV)

def test_TotalAccountMV():
    #GIVEN
    EXPECTED_ACCOUNT_POSITIONS = [
        implementations.positionSolution.position("IBM US Equity", 530),
        implementations.positionSolution.position("TSLA US Equity", 1120),
        implementations.positionSolution.position("NVDA US Equity", 7421)
    ]
    DATA_SOURCE = priceData()
    DATA_SOURCE.clearPriceHistory()
    testObj = implementations.accountSolution.account(EXPECTED_ACCOUNT_POSITIONS, "Test Account")

    #WHEN
    MV = testObj.getCurrentMarketValue()
    EXPECTED_MV = 0
    for pos in EXPECTED_ACCOUNT_POSITIONS:
        EXPECTED_MV += pos.getPosition() * DATA_SOURCE.getSecurityPriceDataList(pos.getSecurity().getName())[-1]
    #EXPECT
    assert (EXPECTED_MV == MV)

#TODO Portfolio MV Tests